In [2]:
import numpy as np
import scipy.linalg as la
from qiskit.visualization import array_to_latex

<div class="alert alert-block alert-info" text-align="center">
<p style="text-align: left; color: navy;">  
<b>Singular Value Decomposition Theorem: SVD</b>  
<br>    
Let $A$ be a complex $m\times n$ matrix. Then it admits the following form (singular value decomposition)  
<br>  
<br>  
$$
A = U \,\Sigma\, V^{\dagger} \,,
$$  
<br>  
where $U \in U(m)$, $V \in U(n)$ are square unitary matrices and $\Sigma$ is a rectangular $m \times n$ matrix with the nonzero singular values $s_1, ..., s_r$ of $A$ on the diagonal, where $r \leq \min(m,n)$.
</div>


<div class="alert alert-block alert-danger">
The following equations hold

\begin{align}
A^\dagger A &= V\,\Sigma^\dagger U\,U^\dagger \Sigma\,V^\dagger = V\,(\Sigma^\dagger \Sigma)\,V^\dagger \\[6pt]
A A^\dagger &= U\,\Sigma V^\dagger V\,\Sigma^\dagger U^\dagger = U\,(\Sigma \Sigma^\dagger)\,U^\dagger
\end{align}

or equivalently

$$
\Sigma^\dagger \Sigma  = V^\dagger A^\dagger A V
\quad,\quad
\Sigma \Sigma^\dagger = U^\dagger A A^\dagger U
$$

The matrices

$$
\Sigma \Sigma^\dagger = \mathrm{diag}(\lambda_1,\dots,\lambda_r,0,\dots,0)_{\,m\times m}
\quad,\quad
\Sigma^\dagger \Sigma = \mathrm{diag}(\lambda_1,\dots,\lambda_r,0,\dots,0)_{\,n\times n}
$$

are diagonal. Therefore:

- The matrix $V$ diagonalizes $A^\dagger A$. Hence its columns are the $n$ eigenvectors of $A^\dagger A$.
- The matrix $U$ diagonalizes $A A^\dagger$. Hence its columns are the $m$ eigenvectors of $A A^\dagger$.

This is an efficient way to find the matrices $U$ and $V$.

Notice that any $\tilde V = VD$ and $\tilde U = UF$ such that

$$
D^\dagger \Sigma^\dagger \Sigma D = \Sigma^\dagger \Sigma~~~~,~~~~~
F^\dagger \Sigma \Sigma^\dagger  F = \Sigma \Sigma^\dagger 
$$
will lead to a different solution of the form 
$$
\Sigma^\dagger \Sigma  = \tilde V^\dagger A^\dagger A \tilde V
\quad,\quad
\Sigma \Sigma^\dagger = \tilde U^\dagger A A^\dagger \tilde U
$$
Now
$$
A = \tilde U \Sigma \tilde V^\dagger = UD\Sigma F^\dagger V^\dagger
$$

</div>


In [5]:
'NumPy has the `svd` function to perform singular value decomposition.'
m=2
n=2

'define a random matrix A so perform the svd on it'
A = np.matrix(np.random.randn(m,n)+ 1j*np.random.randn(m,n))
display(array_to_latex(A,prefix='A='))
print( 'the shape of A is :', A.shape)


u, s, vh = la.svd(A, full_matrices=True)

print( 'the shape of u =',u.shape, ' s =', s.shape,' v =', vh.shape)

<IPython.core.display.Latex object>

the shape of A is : (2, 2)
the shape of u = (2, 2)  s = (2,)  v = (2, 2)


In [6]:
' let us visualize the matrices U and V. For S we will need to pad with 0s'
U=np.matrix(u)
S= np.zeros((m, n))
np.fill_diagonal(S,s)
S = np.matrix(S)
V=np.matrix(vh).getH()

display(array_to_latex(U,prefix='U='))
display(array_to_latex(S,prefix='S='))
display(array_to_latex(V,prefix='V='))

display(array_to_latex(np.round(S@S.getH(),8),prefix='S S^{\dagger} ='))
display(array_to_latex(np.round(S.getH()@S,8),prefix='S^{\dagger}S ='))

#'''Verifiquemos unitariedad'''
#display(array_to_latex(np.dot(V.getH(),V),prefix='V^{\dagger}V ='))
#display(array_to_latex(np.dot(U.getH(),U),prefix='U^{\dagger}U ='))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [8]:
'Perform the multiplication to check out that A =  U * S * V^dagger'
A_recons = U @ S @ V.getH()

'Display the result'
#display(array_to_latex(A_reconstructed, prefix='U S V^\\dagger = '))

np.allclose(A, A_recons)


True

In [9]:
'let us do it by hand. First we diagonalize A^\dagger A'

AdA = A.getH()@A
#display(array_to_latex(AdA,prefix='A^{\dagger}A = '))

'it is important to sort the eigenvalues in descending order'
eigvals_AdA, eigvecs_AdA = np.linalg.eig(AdA)
idx = eigvals_AdA.argsort()[::-1] 
eigvals_AdA, eigvecs_AdA = eigvals_AdA[idx], eigvecs_AdA[:, idx]

'check that the eigenvalues are the same as those of S^\dagger S before'
display(array_to_latex(eigvals_AdA, prefix='\lambda_{AdA} = '))

'the matrix V_AdA is the matrix of eigenvectors of A^{\dagger}A '
V_AdA = np.matrix(eigvecs_AdA)
display(array_to_latex(V_AdA, prefix='V_{AdA} = '))

'check'
display(array_to_latex(V_AdA.getH()@ AdA @ V_AdA,prefix='V_{AdA}^{\dagger} (A^\dagger A) V_{AdA} ='))
display(array_to_latex(S.getH()@ S,prefix='S^{\dagger} S ='))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [10]:
'to find U we diagonalize A A^\dagger'
AAd=  A@A.getH()

'it is important to sort the eigenvalues in descending order'
eigvals_AAd, eigvecs_AAd = np.linalg.eig(AAd)
idx = eigvals_AAd.argsort()[::-1] 
eigvals_AAd, eigvecs_AAd = eigvals_AAd[idx], eigvecs_AAd[:, idx]

'check that the eigenvalues are the same as those of S^\dagger S before'
display(array_to_latex(eigvals_AAd, prefix='\lambda_{AAd} = '))

'the matrix U_AdA is the matrix of eigenvectors of A A^{\dagger}'
U_AAd = np.matrix(eigvecs_AAd)
display(array_to_latex(U_AAd, prefix='U_{AAd} = '))

'check'
display(array_to_latex(U_AAd.getH()@ AAd @ U_AAd,prefix='U_{AAd}^{\dagger} (A A^\dagger) U_{AAd} ='))
display(array_to_latex(U.getH()@ AAd @ U,prefix='U^{\dagger} (A A^\dagger) U ='))

display(array_to_latex(S@S.getH(),prefix='S S^{\dagger}='))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [14]:
'Finally we perform the multiplication U_AAd * S * V_AdA^dagger'
A_recons2 = U_AAd @ S @ V_AdA.getH()

'Display the result'
display(array_to_latex(A_recons2, prefix='U_{AAd} S V_{AdA}^\\dagger = '))

'the result fails to give the correct A'
display(array_to_latex(A, prefix='A = '))
np.allclose(A, A_recons2)


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

False

The matrices $U$ and $U_{AAd}$ differ by a diagonal matrix $D$ such that $U_{AAd} = U D$ 
which fulfills

$$
D (S S^\dagger) D^\dagger  =   S S^\dagger
$$

so they are an allowed ambiguity. 

In [16]:
'The matrices U and U_AAd differ by a diagonal matrix   U_AAd = U D'
D = U.getH()@U_AAd

display(array_to_latex(D,prefix='D = U_{AAd}^{\dagger}U  ='))

'The matrix $D$ is a diagonal matrix of phases'
display(array_to_latex(D.getH()@D))


'such that D S S^+ D^+ = S S^+'
display(array_to_latex(D@S@S.getH()@D.getH() - S@S.getH(),prefix='D S S^{\dagger}D^\dagger - S S^{\dagger} ='))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

The matrices $V$ and $V_{AdA}$ differ by a diagonal matrix $F$ such that  $V_{AdA} = V F$ 
which fulfills

$$
F (S^\dagger S) F^\dagger  =   S^\dagger S
$$

so they are an allowed ambiguity. 

In [17]:
'The matrices V and V_AdA  differ by a diagonal matrix F'
F = V.getH()@V_AdA

display(array_to_latex(F,prefix='F = V_{AdA}^{\dagger} V ='))

'such that F S^+ S D^+ = S^+ S'
display(array_to_latex(F@S.getH()@S@F.getH() -S.getH()@S,prefix='F S^{\dagger}S F^\dagger - F^{\dagger}F ='))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Hhowever we do not get that $A = U_{AAd} S V_{AdA}^+$ because this ambiguity is not one of the SVD of A. 
Indeed

$$
U_{AAd} S V_{AdA}^+ = UD S F^\dagger V^\dagger =  U S V^+
$$

would need
$$
D S F^\dagger = S
$$

which is not satisfied.

In [18]:
'however we do not get that U_AAd S V_AdA^+ = U  S V^+ because DSF^+ is not equal to S'
display(array_to_latex(D@S@F.getH() - S,prefix='D S F^\dagger - S  ='))


<IPython.core.display.Latex object>